In [3]:
#import packages
#os to fast track terminal use
import os
os.system("pip install pandas spacy")
#spacy for nlp
import spacy
os.system("python -m spacy download en_core_web_md")
nlp = spacy.load('en_core_web_md')
#pandas for dataframes
import pandas as pd
#numpy for hjelp
import numpy as np

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 22.4 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
/tmp/ipykernel_700/454231508.py:10: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
#get file
#filepath set for sparately added Ucloud data directory
fp = "/work/cds-lang-data/USEcorpus/USEcorpus/"
#os independent filepath

#filepath for hand-in?


In [49]:
#sorted(os.listdir(filepath))
# function for 1 file from 1 folder

def count_ofof(fp, extract_this = ["NOUN","VERB","ADJ","ADV"]):
    #load it with encoding 
    with open(fp, encoding="latin-1") as f:
        text = f.read()

    #convert it with spacy

    text_nlp = nlp(text)

    # extract values, filter  NOUN, VERB, ADJ, ADV I have to use "i" to  index items, long words confuse me
    extract_this = ["NOUN","VERB","ADJ","ADV"]
    #empty list for extracts
    extract = []
    for i in text_nlp:
        if i.pos_ in extract_this:
            extract.append(i.pos_)
        else:
            pass 
    #cunt unique values
    count_extracts = np.unique(extract, return_counts= True )
    return(count_extracts)


In [78]:
#turn it to dictionary, because I want to practice working with those
def dict_it(count_extracts):
    dict_extracts = {}
    dict_list =[]
    for i in range(len(count_extracts[1])):
        if i == 0:
            dict_extracts["pos"] = count_extracts[0][0]
            dict_extracts["count"] = count_extracts[1][0]
            dict_list.append(dict_extracts)
        else:
            upd_ext = {"pos":count_extracts[0][i],"count":count_extracts[1][i]}
            dict_list.append(upd_ext)
    return(dict_list)


In [85]:
# multiple files from 1 folder
fp = "/work/cds-lang-data/USEcorpus/USEcorpus/a1/"
fp_s = sorted(os.listdir(fp))

def count_mfof(fp):
    fp_s = sorted(os.listdir(fp))
    dict_list_list = []
    #get count out of the file
    for i in fp_s: 
        extract_array = count_ofof(fp + i)
        extract_dict = dict_it(extract_array)
        #add filename to each dict in list to prevent confusion on where the data came from
        for j in extract_dict:
            j.update({"file":i})
    
        dict_list_list.append(extract_dict)
    return(dict_list_list)    

test = count_mfof(fp)


In [ ]:
len(test)

[[{'pos': 'ADJ', 'count': 57, 'file': '0100.a1.txt'},
  {'pos': 'ADV', 'count': 38, 'file': '0100.a1.txt'},
  {'pos': 'NOUN', 'count': 109, 'file': '0100.a1.txt'},
  {'pos': 'VERB', 'count': 87, 'file': '0100.a1.txt'}],
 [{'pos': 'ADJ', 'count': 47, 'file': '0101.a1.txt'},
  {'pos': 'ADV', 'count': 67, 'file': '0101.a1.txt'},
  {'pos': 'NOUN', 'count': 93, 'file': '0101.a1.txt'},
  {'pos': 'VERB', 'count': 99, 'file': '0101.a1.txt'}],
 [{'pos': 'ADJ', 'count': 57, 'file': '0102.a1.txt'},
  {'pos': 'ADV', 'count': 40, 'file': '0102.a1.txt'},
  {'pos': 'NOUN', 'count': 124, 'file': '0102.a1.txt'},
  {'pos': 'VERB', 'count': 100, 'file': '0102.a1.txt'}],
 [{'pos': 'ADJ', 'count': 54, 'file': '0103.a1.txt'},
  {'pos': 'ADV', 'count': 52, 'file': '0103.a1.txt'},
  {'pos': 'NOUN', 'count': 99, 'file': '0103.a1.txt'},
  {'pos': 'VERB', 'count': 123, 'file': '0103.a1.txt'}],
 [{'pos': 'ADJ', 'count': 46, 'file': '0104.a1.txt'},
  {'pos': 'ADV', 'count': 55, 'file': '0104.a1.txt'},
  {'pos': 'N